In [3]:
import pandas as pd
# import geopandas as gpd
# import arrow
import altair as alt

## Import data

In [10]:

# Source = https://data.ca.gov/dataset/water-quality-data/resource/82916200-3700-4e3d-83d5-0a3ac6dc31a8
file_name = 'field_results'
# today = arrow.utcnow().format('MMDDYY')

def import_df(file_name):
	# print(today)
	try:
		df = pd.read_parquet(f'{file_name}.parquet')
	except Exception as e:
		# print(e)
		print("Creating Feather file")
		df = pd.read_csv(f'{file_name}.csv')
		# df.to_parquet(f'{file_name}_{today}.parquet')
		df.to_parquet(f'{file_name}.parquet')

	return df
full_df = import_df(file_name)

## Select Time Frame

In [20]:
def trim_dates(df,years):
	df_trim = df.loc[pd.to_datetime(df['SAMPLE_DATE']).dt.year.isin(years)]
	# https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
	df_trim['Month'] = pd.to_datetime(df_trim['SAMPLE_DATE']).dt.strftime('%Y-%m')
	# df_trim['Month'] = pd.to_datetime(df_trim['SAMPLE_DATE']).dt
	return df_trim

df_trim = trim_dates(full_df,[2018,2019,2020,2021,2022])
df = df_trim
print(df.shape)
df.head(2)

(42502, 24)


,STATION_ID,STATION_NAME,STATION_NUMBER,FULL_STATION_NAME,STATION_TYPE,LATITUDE,LONGITUDE,STATUS,COUNTY_NAME,SAMPLE_CODE,...,ANL_DATA_TYPE,PARAMETER,FDR_RESULT,FDR_TEXT_RESULT,FDR_DATE_RESULT,FDR_REPORTING_LIMIT,UNS_NAME,MTH_NAME,FDR_FOOTNOTE,Month
5848,12,H.O. Banks Headworks,KA000331,Delta P.P. Headworks at H.O. Banks PP,Surface Water,37.8019,-121.62,Review Status Unknown,Alameda,D0118B0001,...,None,SpecificConductance,550.0,None,None,1.0,uS/cm@25 °C,EPA 120.1 (Field),None,2018-01
5849,12,H.O. Banks Headworks,KA000331,Delta P.P. Headworks at H.O. Banks PP,Surface Water,37.8019,-121.62,Review Status Unknown,Alameda,D0118B0001,...,None,WaterTemperature,10.9,None,None,0.1,°C,EPA 170.1 (Field),None,2018-01


## Set Criteria

In [21]:
df = df_trim.loc[
	(df_trim['UNS_NAME'] == 'mg/L') & 
	(df_trim['SAMPLE_DEPTH_UNITS'] == 'Feet') & 
	(df_trim['SAMPLE_DEPTH'] == 3) & 
	(df_trim['PARAMETER'] == 'DissolvedOxygen' )]

my_headers = ['STATION_NAME','PARAMETER','FDR_RESULT','UNS_NAME','SAMPLE_DATE','SAMPLE_DEPTH','SAMPLE_DEPTH_UNITS','Month','COUNTY_NAME']
df[my_headers].head(2)

,STATION_NAME,PARAMETER,FDR_RESULT,UNS_NAME,SAMPLE_DATE,SAMPLE_DEPTH,SAMPLE_DEPTH_UNITS,Month,COUNTY_NAME
24239,"Lake Oroville, Dam",DissolvedOxygen,6.63,mg/L,2021-12-15 09:30:00,3.0,Feet,2021-12,Butte
24251,"Lake Oroville, Dam",DissolvedOxygen,10.50,mg/L,2022-01-19 08:24:00,3.0,Feet,2022-01,Butte


In [22]:
df.to_parquet('DO_data.parquet')

## Visualize

In [10]:
base = alt.Chart(df).properties(width=700,height=300)

selection = alt.selection_multi(fields=['STATION_NAME'], bind='legend')

dots = base.mark_point(size=30).encode(
    # x=alt.X('Month',title='Month'),
    x=alt.X('SAMPLE_DATE',title='Month'),
    y=alt.Y('FDR_RESULT',title='Dissolved Oxygen (mg/L)'),
    shape=alt.Shape('COUNTY_NAME',title='County'),
    color=alt.Color('STATION_NAME',title='Station Name'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
    size=alt.condition(selection, alt.value(50), alt.value(5)),
    tooltip=my_headers
).add_selection(selection).interactive()
dots


alt.Chart(...)